In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [ ]:
import numpy as np
import librosa

from composition.instrument import Score, Part, Phrase, show
import composition.common as k
from utils import QuantileTransformer

In [ ]:
def fib(n):
    first = 0
    second = 1
    fibs = []
    for _ in range(n):
        first, second = second, first + second
        fibs.append(first)
    
    return fibs

In [ ]:
fib(15)

In [ ]:
phi = (np.sqrt(5) + 1) / 2
dur = 7 * 60
b = dur / phi
a = dur - b
a2 = a / phi
a1 = a - a2

b1 = b / phi
b2 = b - b1

print(f'A: {a1:.2f}\" B: {a2:.2f}\" development: {b1:.2f}\" A\'/B\': {b2:.2f}\"')

## Sections

|           | A   | B   | D    | R   | Total |
| --------- | --- | --- | ---- | --- | ----- |
| Rounded   | 61" | 99" | 160" | 99" | 7'    |
| Fibonacci | 55" | 89" | 144" | 89" | 6'17" |

* violin: mean: -43.36, std: 5.23
* viola: mean: -42.76, std: 7.77
* cello: mean: -57.26, std: 6.12
* flute: mean: -55.27, std: 8.37
* flute2: mean: -55.27, std: 8.37
- Violin 55-103
- Viola 48-91
- Cello 36-76
- Flute 60-96
- Flute2 55-91

In [ ]:
dur = 16

In [ ]:
seed_p = np.array((0, 0.7, 0.3, 1.0))
seed_a = np.array((0, 0.375, 0.625, 1.))

In [ ]:
COPPER = 4.236067978

In [ ]:
fig, ax = plt.subplots(1, figsize=(4 * COPPER, 4))
ax.plot(seed_a, seed_p)
plt.show()

In [ ]:
vln1 = Part('Vln. 1', 'violin')
for i, dr in enumerate([2, 4, 8]):
    seed_d = k.to_delta(seed_a * dr)
    pitches, durs = k.fractal_bpc_floor(seed_p, seed_d, i)
    pitches = np.pad(pitches, (0, k.time_to_step(dr) - len(pitches)), constant_values=pitches[-1])
    pitches = (pitches - min(pitches)) / (max(pitches) - min(pitches))
    pitches = k.rescale(pitches, 66, 66+12)
    pitches = k.autotune(pitches, 0.95)
    pitches += k.sinusoid(dr, 4, 0, 0.125)
    
    if i < 2:
        amp_segments = []
        for d in durs:
            aa = k.triangle(d, 1/d, 1e-7)
            aa = np.pad(aa, (0, k.time_to_step(d) - len(aa)), constant_values=aa[-1])
            amp_segments.append(aa)
        amps = np.concatenate(amp_segments)
        amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
        amps = k.rescale(amps, -55, -44)
        amps += k.line_segment(dr, -12, 4)
    else:
        amps = k.line_segment(dr, 0, 1)
        amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
        amps = k.rescale(amps, -59, -44)
    
    amps += np.random.randn(*amps.shape) * 0.5
    vln1.add_phrase(Phrase(pitches, amps))
    
    breathe_p = k.constant(1 + i, pitches[-1])
    vib = k.sinusoid(1+i, 4, 0, 0.125) * (k.phasor(1+i, 1/(1+i)) ** 2)
    breathe_p += vib
    if i == 2:
        breath_amp = (1 - k.phasor(1+i, 1/(1+i)))
        breath_amp = k.rescale(breath_amp, -58, -46)
        breath_amp[-1] = breath_amp[-2]
    else:
        breath_amp = k.line_segment(1+i, 0, 1) * 9 - 55
    
    breath_amp += np.random.randn(*breath_amp.shape) * 0.5
    vln1.add_phrase(Phrase(breathe_p, breath_amp))

In [ ]:
vln1.show()
vln1.play()

In [ ]:
vln2 = Part('Vln. 2', 'violin')
for i, dr in enumerate([2, 4, 8]):
    seed_d = k.to_delta(seed_a * dr)
    pitches, durs = k.fractal_bpc_floor(seed_p[::-1], np.roll(seed_d, 1), i)
    pitches = np.pad(pitches, (0, k.time_to_step(dr) - len(pitches)), constant_values=pitches[-1])
    pitches = (pitches - min(pitches)) / (max(pitches) - min(pitches))
    pitches = k.rescale(pitches, 61, 61+12)
    pitches = k.autotune(pitches, 0.95)
    pitches += k.sinusoid(dr, 4, 0, 0.125)
    
    if i < 2:
        amp_segments = []
        for d in durs:
            aa = k.triangle(d, 1/d, 1e-7)
            aa = np.pad(aa, (0, k.time_to_step(d) - len(aa)), constant_values=aa[-1])
            amp_segments.append(aa)
        amps = np.concatenate(amp_segments)
        amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
        amps = k.rescale(amps, -55, -44)
        amps += k.line_segment(dr, -12, 4)
    else:
        amps = k.line_segment(dr, 0, 1)
        amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
        amps = k.rescale(amps, -59, -44)
    
    amps += np.random.randn(*amps.shape) * 0.5
    vln2.add_phrase(Phrase(pitches, amps))
    
    breathe_p = k.constant(1 + i, pitches[-1])
    vib = k.sinusoid(1+i, 4, 0, 0.125) * (k.phasor(1+i, 1/(1+i)) ** 2)
    breathe_p += vib
    if i == 2:
        breath_amp = (1 - k.phasor(1+i, 1/(1+i)))
        breath_amp = k.rescale(breath_amp, -58, -46)
        breath_amp[-1] = breath_amp[-2]
    else:
        breath_amp = k.phasor(1+i, 1/(1+i)) * 9 - 55
    
    breath_amp += np.random.randn(*breath_amp.shape) * 0.5
    vln2.add_phrase(Phrase(breathe_p, breath_amp))

In [ ]:
vln2.show()
vln2.play()

In [ ]:
flt1 = Part('Flt. 1', 'flute')
silence = Phrase(k.constant(17, 66), k.constant(17, -110))
flt1.add_phrase(silence)
for i, dr in enumerate([1, 2, 4]):
    seed_d = k.to_delta(seed_a * dr)
    pitches, durs = k.fractal_bpc_floor(np.roll(seed_p, 1), np.roll(seed_d, 2), i)
    pitches = np.pad(pitches, (0, k.time_to_step(dr) - len(pitches)), constant_values=pitches[-1])
    pitches = (pitches - min(pitches)) / (max(pitches) - min(pitches))
    pitches = k.rescale(pitches, 71, 71+12)
    # pitches = k.autotune(pitches, 0.95, list(np.random.randint(0, 11, 5)))
    pitches = k.autotune(pitches, 0.95)
    pitches += k.sinusoid(dr, 4, 0, 0.125)

    # amps = k.sinusoid(dr, 2/dr, -43.36, 5.23/2)
    amps = np.concatenate([k.triangle(d, 1/d, 0.01) for d in durs])
    # amps += np.random.normal(0, .5, len(amps))
    # amps = k.to_db_loudness(amps, 'violin', min_loudness=-0.1)
    amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
    amps = k.rescale(amps, -55, -54)
    amps += k.line_segment(dr, 0, 6)
    
    amps += np.random.randn(*amps.shape) * 0.2
    flt1.add_phrase(Phrase(pitches, amps))
    flt1.add_phrase(Phrase(pitches[::-1], amps))
    
    if i == 2:
        i = 3
    breathe_p = k.constant(1 + i, pitches[-1])
    vib = k.sinusoid(1+i, 4, 0, 0.125) * (k.phasor(1+i, 1/(1+i)) ** 2)
    breathe_p += vib
    breath_amp = k.phasor(1+i, 1/(1+i)) * 9 - 55
    
    breath_amp += np.random.randn(*breath_amp.shape) * 0.2
    flt1.add_phrase(Phrase(breathe_p, breath_amp))

In [ ]:
flt1.show(17)
flt1.play(17)

In [ ]:
flt2 = Part('Flt. 2', 'flute2')
silence = Phrase(k.constant(17, 66), k.constant(17, -110))
flt2.add_phrase(silence)
for i, dr in enumerate([1, 2, 4]):
    seed_d = k.to_delta(seed_a * dr)
    pitches, durs = k.fractal_bpc_floor(np.roll(seed_p, 2), np.roll(seed_d, 3), i)
    pitches = np.pad(pitches, (0, k.time_to_step(dr) - len(pitches)), constant_values=pitches[-1])
    pitches = (pitches - min(pitches)) / (max(pitches) - min(pitches))
    pitches = k.rescale(pitches, 66, 66+12)
    # pitches = k.autotune(pitches, 0.95, list(np.random.randint(0, 11, 5)))
    pitches = k.autotune(pitches, 0.95)
    pitches += k.sinusoid(dr, 4, 0, 0.125)

    # amps = k.sinusoid(dr, 2/dr, -43.36, 5.23/2)
    amps = np.concatenate([k.triangle(d, 1/d, 0.01) for d in durs])
    # amps += np.random.normal(0, .5, len(amps))
    # amps = k.to_db_loudness(amps, 'violin', min_loudness=-0.1)
    amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
    amps = k.rescale(amps, -58, -57)
    amps += k.line_segment(dr, 0, 6)
    
    amps += np.random.randn(*amps.shape) * 0.2
    flt2.add_phrase(Phrase(pitches, amps))
    flt2.add_phrase(Phrase(pitches[::-1], amps))
    
    if i == 2:
        i = 3
    breathe_p = k.constant(1 + i, pitches[-1])
    vib = k.sinusoid(1+i, 4, 0, 0.125) * (k.phasor(1+i, 1/(1+i)) ** 2)
    breathe_p += vib
    breath_amp = k.phasor(1+i, 1/(1+i)) * 9 - 55
    
    breath_amp += np.random.randn(*breath_amp.shape) * 0.2
    flt2.add_phrase(Phrase(breathe_p, breath_amp))

In [ ]:
flt2.show(17)
flt2.play(17)

In [ ]:
volume = -4

vlc = Part('Vlc', 'cello')
for i, dr in enumerate([2, 4, 8]):
    seed_d = k.to_delta(seed_a * dr)
    pitches, durs = k.fractal_bpc_floor(seed_p, seed_d, i)
    pitches = np.pad(pitches, (0, k.time_to_step(dr) - len(pitches)), constant_values=pitches[-1])
    pitches = (pitches - min(pitches)) / (max(pitches) - min(pitches))
    pitches = k.rescale(pitches, 42, 42+12)
    pitches = k.autotune(pitches, 0.95)
    pitches += k.sinusoid(dr, 4, 0, 0.125)
    
    if i < 2:
        amps = np.concatenate([k.triangle(d, 1/d, 0.01) for d in durs])
        amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
        amps = k.rescale(amps, -55, -44)
        amps += k.line_segment(dr, -12, 4)
    else:
        amps = k.line_segment(dr, 0, 1)
        amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
        amps = k.rescale(amps, -59, -44)
    
    amps += volume
    amps += np.random.randn(*amps.shape) * 0.5
    vlc.add_phrase(Phrase(pitches, amps))
    
    breathe_p = k.constant(1 + i, pitches[-1])
    vib = k.sinusoid(1+i, 4, 0, 0.125) * (k.phasor(1+i, 1/(1+i)) ** 2)
    breathe_p += vib
    
    if i == 2:
        breath_amp = (1 - k.phasor(1+i, 1/(1+i))) * 9 - 55
        breath_amp = k.rescale(breath_amp, -59, -46)
        breath_amp[-1] = breath_amp[-2]
    else:
        breath_amp = k.phasor(1+i, 1/(1+i)) * 9 - 55
    
    breath_amp += volume
    breath_amp += np.random.randn(*breath_amp.shape) * 0.5
    vlc.add_phrase(Phrase(breathe_p, breath_amp))

In [ ]:
vlc.show()
vlc.play()

In [ ]:
volume = 4

vla = Part('Vla', 'viola')
for i, dr in enumerate([2, 4, 8]):
    seed_d = k.to_delta(seed_a * dr)
    pitches, durs = k.fractal_bpc_floor(seed_p[::-1], np.roll(seed_d, 1), i)
    pitches = np.pad(pitches, (0, k.time_to_step(dr) - len(pitches)), constant_values=pitches[-1])
    pitches = (pitches - min(pitches)) / (max(pitches) - min(pitches))
    pitches = k.rescale(pitches, 49, 49+12)
    # pitches = k.autotune(pitches, 0.95, list(np.random.randint(0, 11, 5)))
    pitches = k.autotune(pitches, 0.95)
    pitches += k.sinusoid(dr, 4, 0, 0.125)
    
    if i < 2:
        # amps = k.sinusoid(dr, 2/dr, -43.36, 5.23/2)
        amps = np.concatenate([k.triangle(d, 1/d, 0.01) for d in durs])
        # amps += np.random.normal(0, .5, len(amps))
        # amps = k.to_db_loudness(amps, 'violin', min_loudness=-0.1)
        amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
        amps = k.rescale(amps, -55, -44)
        amps += k.line_segment(dr, -12, 4)
        amps = k.rescale(amps, -60, -48)
    else:
        amps = k.line_segment(dr, 0, 1)
        amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
        amps = k.rescale(amps, -60, -48)
    
    amps += volume
    amps += np.random.randn(*amps.shape) * 0.5
    vla.add_phrase(Phrase(pitches, amps))
    
    breathe_p = k.constant(1 + i, pitches[-1])
    vib = k.sinusoid(1+i, 4, 0, 0.125) * (k.phasor(1+i, 1/(1+i)) ** 2)
    breathe_p += vib
    
    if i == 2:
        breath_amp = (1 - k.phasor(1+i, 1/(1+i))) * 9 - 55
        breath_amp = k.rescale(breath_amp, -66, -47)
        breath_amp[-1] = breath_amp[-2]
    else:
        breath_amp = k.phasor(1+i, 1/(1+i)) * 9 - 55
        breath_amp = k.rescale(breath_amp, -55, -47)

    breath_amp += volume
    breath_amp += np.random.randn(*breath_amp.shape) * 0.5
    vla.add_phrase(Phrase(breathe_p, breath_amp))

In [ ]:
vla.show()
vla.play()

In [ ]:
# silence, self, retro, chord, self, retro, chord
# skip slience, first self, first retro
_, idx = np.unique(flt1.phrases[4].pitch.round(), return_index=True)
chord = flt1.phrases[4].pitch.round()[np.sort(idx)]
for idx in range(7):
    dur = 2
    pitches = k.constant(dur, chord[(2*idx)%len(chord)])
    attack = k.triangle(dur, 1/dur, ratio=0.01)
#     silence = k.constant(dur/2, 0)
#     amps = np.concatenate([attack, silence])
    amps = k.rescale(attack, -90, -42)
    if idx == 2:
        amps = k.line_segment(dur, -58, -46)
    
    phrase = Phrase(pitches, amps)
    vln1.add_phrase(phrase)

amps = k.line_segment(4, -58, -42)
pitches = k.constant(4, chord[(2*(idx+1))%len(chord)])
phrase = Phrase(pitches, amps)
vln1.add_phrase(phrase)

In [ ]:
vln1.show()
vln1.play()

In [ ]:
# silence, self, retro, chord, self, retro, chord
# skip slience, first self, first retro
_, idx = np.unique(flt1.phrases[4].pitch.round(), return_index=True)
chord = flt1.phrases[4].pitch.round()[np.sort(idx)]
for idx in range(7):
    dur = 2
    pitches = k.constant(dur, chord[(3*idx + 1)%len(chord)])
    attack = k.triangle(dur, 1/dur, ratio=0.01)
#     silence = k.constant(dur/2, 0)
#     amps = np.concatenate([attack, silence])
    amps = k.rescale(attack, -90, -42)
    if idx == 2:
        amps = k.line_segment(dur, -58, -46)
    
    phrase = Phrase(pitches, amps)
    vln2.add_phrase(phrase)

amps = k.line_segment(4, -58, -42)
pitches = k.constant(4, chord[(3*(idx + 1) + 1)%len(chord)])
phrase = Phrase(pitches, amps)
vln2.add_phrase(phrase)

In [ ]:
vln2.show()
vln2.play()

In [ ]:
# silence, self, retro, chord, self, retro, chord
# skip slience, first self, first retro
_, idx = np.unique(flt1.phrases[4].pitch.round(), return_index=True)
chord = flt1.phrases[4].pitch.round()[np.sort(idx)]
for idx in range(7):
    dur = 2
    pitches = k.constant(dur, chord[(5*idx + 2)%len(chord)] - 24)
    pitches[pitches < 48] += 12
    attack = k.triangle(dur, 1/dur, ratio=0.01)
#     silence = k.constant(dur/2, 0)
#     amps = np.concatenate([attack, silence])
    amps = k.rescale(attack, -90, -38)
    if idx == 2:
        amps = k.line_segment(dur, -58, -42)
    
    phrase = Phrase(pitches, amps)
    vla.add_phrase(phrase)

amps = k.line_segment(4, -58, -42)
pitches = k.constant(4, chord[(5*(idx+1) + 2)%len(chord)] - 24)
pitches[pitches < 48] += 12
phrase = Phrase(pitches, amps)
vla.add_phrase(phrase)

In [ ]:
vla.show(20)
vla.play(20)

In [ ]:
# silence, self, retro, chord, self, retro, chord
# skip slience, first self, first retro
_, idx = np.unique(flt1.phrases[4].pitch.round(), return_index=True)
chord = flt1.phrases[4].pitch.round()[np.sort(idx)]
for idx in range(7):
    dur = 2
    pitches = k.constant(dur, chord[(3*idx + 3)%len(chord)] - 36)
    pitches[pitches < 36] += 12
    attack = k.triangle(dur, 1/dur, ratio=0.01)
#     silence = k.constant(dur/2, 0)
#     amps = np.concatenate([attack, silence])
    amps = k.rescale(attack, -90, -38)
    if idx == 2:
        amps = k.line_segment(dur, -58, -42)
    
    phrase = Phrase(pitches, amps)
    vlc.add_phrase(phrase)

amps = k.line_segment(4, -58, -42)
pitches = k.constant(4, chord[(3*(idx+1) + 3)%len(chord)] - 36)
pitches[pitches < 36] += 12
phrase = Phrase(pitches, amps)
vlc.add_phrase(phrase)

In [ ]:
vlc.show(20)
vlc.play(20)

In [ ]:
score = Score([vln1, vln2, vla, vlc, flt1, flt2])

In [ ]:
score.save('fractus-duo', './audio-data/december/')

In [ ]:
def pad_to_dur(array, dur):
    padding = k.time_to_step(dur) - len(array)
    if padding < 0:
        return array[:padding]
    return np.pad(array, (0, padding), constant_values=array[-1])

In [ ]:
def fib(n, first=None, second=None):
    if not first:
        first = [1]
    if not second:
        second = [2]
    
    while len(second) < n:
        first, second = second, second + first

    return np.array(second[:n])

flutler 200 adim fracatal level 1'de = 12 saniye
fractal level 3 kemanlarin cumlesi 82 adim yaklasik 8 saniye

In [ ]:
flt1.phrases[-1].pitch[-1], flt2.phrases[-1].pitch[-1]

In [ ]:
vln1.phrases[-1].pitch[-1], vln2.phrases[-1].pitch[-1]

In [ ]:
new_seed = np.array([0.0, 0.7, 0.3, 1.0])
new_seed2 = np.array([0.0, 0.8, 0.4, 1.0])

In [ ]:
for i in range(4):
    pitches = k.fractal(new_seed, i)
    print(len(pitches))

In [ ]:
ritardando = np.linspace(0.06, 0.2, 40)

In [ ]:
part3 = Part('tmp3', 'flute')
# 10 pitches
n_steps = 10*4
pitches = k.fractal(new_seed[::-1], 1)
pitches = np.tile(pitches, 1 + n_steps//len(pitches))[:n_steps]
durs = fib(len(pitches))
durations = durs * 0.06
dur = sum(durations)
silence_dur = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 66, 66+24)
pitches = pad_to_dur(pitches, dur)

amps = k.constant(dur, -47)

phrase = Phrase(pitches, amps)
part3.add_phrase(phrase)

n_steps = 82
pitches = k.fractal(new_seed[::-1], 1)
pitches = np.tile(pitches, 1 + n_steps//len(pitches))[:n_steps]
durs = fib(len(pitches))
durations = durs * 0.06
dur = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 66, 66+24)
pitches = pad_to_dur(pitches, dur)

amps = k.constant(dur, -51)

phrase = Phrase(pitches, amps)
part3.add_phrase(phrase)

n_steps = 28*2
pitches = k.fractal(new_seed2[::-1], 2)
pitches = np.tile(pitches, 1 + n_steps//len(pitches))[:n_steps]
durs = fib(len(pitches))
durations = durs * 0.06
dur = sum(durations)
silence_dur2 = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 64, 64+24)
pitches = pad_to_dur(pitches, dur)

amps = k.constant(dur, -47)

phrase = Phrase(pitches, amps)
part3.add_phrase(phrase)

n_steps = 82
pitches = k.fractal(new_seed2[::-1], 1)
pitches = np.tile(pitches, 1 + n_steps//len(pitches))[:n_steps]
durs = fib(len(pitches))
durations = durs * 0.06
dur = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 64, 64+24)
pitches = pad_to_dur(pitches, dur)

amps = k.constant(dur, -51)

phrase = Phrase(pitches, amps)
part3.add_phrase(phrase)

n_steps = 10*4
pitches = k.fractal(new_seed[::-1], 1)
pitches = np.tile(pitches, 1 + n_steps//len(pitches))[:n_steps]
durs = fib(len(pitches))
durations = durs * ritardando
dur = sum(durations)

pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 64, 64+24)
pitches = pad_to_dur(pitches, dur)
pitches += k.sinusoid(dur, 4, 0, 0.25, np.random.uniform())

amps = k.constant(dur, -47)
human = np.random.normal(0, 0.125, len(amps)-1)
human = k.from_delta(human)
amps += human

phrase = Phrase(pitches, amps)
part3.add_phrase(phrase)

In [ ]:
part3.show()
part3.play(25)

In [ ]:
part4 = Part('tmp4', 'flute2')
# 10 pitches
n_steps = 10 * 4
pitches = k.fractal(new_seed, 1)
pitches = np.tile(pitches, 1 + n_steps//len(pitches))[:n_steps]
durs = fib(len(pitches))
durs = np.roll(durs, 1)
durations = durs * 0.06
dur = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 68, 68+24)

amps = k.constant(dur, -44)

pitches = pad_to_dur(pitches, dur)

phrase = Phrase(pitches, amps)
part4.add_phrase(phrase)

n_steps = 82
pitches = k.fractal(new_seed, 1)
pitches = np.tile(pitches, 1 + n_steps//len(pitches))[:n_steps]
durs = fib(len(pitches))
durs = np.roll(durs, 1)
durations = durs * 0.06
dur = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 68, 68+24)

amps = k.constant(dur, -50)

pitches = pad_to_dur(pitches, dur)

phrase = Phrase(pitches, amps)
part4.add_phrase(phrase)

n_steps = 28 * 2
pitches = k.fractal(new_seed2, 2)
pitches = np.tile(pitches, 1 + n_steps//len(pitches))[:n_steps]
durs = fib(len(pitches))
durs = np.roll(durs, 1)
durations = durs * 0.06
dur = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 70, 70+24)

amps = k.constant(dur, -44)

pitches = pad_to_dur(pitches, dur)

phrase = Phrase(pitches, amps)
part4.add_phrase(phrase)

n_steps = 82
pitches = k.fractal(new_seed2, 1)
pitches = np.tile(pitches, 1 + n_steps//len(pitches))[:n_steps]
durs = fib(len(pitches))
durs = np.roll(durs, 1)
durations = durs * 0.06
dur = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 70, 70+24)

amps = k.constant(dur, -50)

pitches = pad_to_dur(pitches, dur)

phrase = Phrase(pitches, amps)
part4.add_phrase(phrase)

n_steps = 10 * 4
pitches = k.fractal(new_seed, 1)
pitches = np.tile(pitches, 1 + n_steps//len(pitches))[:n_steps]
durs = fib(len(pitches))
durs = np.roll(durs, 1)
durations = durs * ritardando
dur = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 70, 70+24)
pitches = pad_to_dur(pitches, dur)
pitches += k.sinusoid(dur, 4, 0, 0.25, np.random.uniform())

amps = k.constant(dur, -44)
human = np.random.normal(0, 0.125, len(amps)-1)
human = k.from_delta(human)
amps += human

phrase = Phrase(pitches, amps)
part4.add_phrase(phrase)

In [ ]:
part4.show()
part4.play(25)

In [ ]:
part = Part('tmp', 'violin')

silence_steps = int(silence_dur*k.SECOND)
# part.add_phrase(Phrase(np.zeros(silence_steps)+55, np.zeros(silence_steps)-100))
pitches = k.constant(silence_dur, np.round(vln1.phrases[-1].pitch[-1]))
amps = k.triangle(silence_dur, 4/silence_dur, 0.05, -90, -44)
part.add_phrase(Phrase(pitches, amps))

# 82 pitches
pitches = k.fractal(new_seed, 3)
durs = fib(len(pitches))
durations = durs * 0.06
dur = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 55, 55+24)
pitches = pad_to_dur(pitches, dur)

amps = k.constant(dur, -44)

phrase = Phrase(pitches, amps)
part.add_phrase(phrase)

silence_steps = int(silence_dur2*k.SECOND)
# part.add_phrase(Phrase(np.zeros(silence_steps)+55, np.zeros(silence_steps)-100))
pitches = k.constant(silence_dur2, np.round(part.phrases[-1].pitch[-1]))
amps = k.triangle(silence_dur2, 2/silence_dur2, 0.05, -90, -44)
part.add_phrase(Phrase(pitches, amps))

# 82 pitches
new_seed = np.array([0.0, 0.7, 0.3, 1.0])
pitches = k.fractal(new_seed2, 3)
durs = fib(len(pitches))
durations = durs * 0.06
dur = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 57, 57+24)
pitches = pad_to_dur(pitches, dur)

amps = k.constant(dur, -44)

phrase = Phrase(pitches, amps)
part.add_phrase(phrase)

n_steps = 10 * 4
pitches = k.fractal(new_seed, 1)
pitches = np.tile(pitches, 1 + n_steps//len(pitches))[:n_steps]
durs = fib(len(pitches))
durs = np.roll(durs, 1)
durations = durs * ritardando
dur = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 57, 57+24)
pitches = pad_to_dur(pitches, dur)
pitches += k.sinusoid(dur, 4, 0, 0.25, np.random.uniform())

amps = k.constant(dur, -44)
human = np.random.normal(0, 0.125, len(amps)-1)
human = k.from_delta(human)
amps += human

phrase = Phrase(pitches, amps)
part.add_phrase(phrase)

In [ ]:
part.show()
part.play(25)

In [ ]:
part2 = Part('tmp2', 'violin')

silence_steps = int(silence_dur*k.SECOND)
# part2.add_phrase(Phrase(np.zeros(silence_steps)+55, np.zeros(silence_steps)-100))
pitches = k.constant(silence_dur, np.round(vln2.phrases[-1].pitch[-1]))
amps = k.triangle(silence_dur, 4/silence_dur, 0.2, -90, -44)
part2.add_phrase(Phrase(pitches, amps))

# 82 pitches
pitches = k.fractal(new_seed[::-1], 3)
durs = fib(len(pitches))
durs = np.roll(durs, 1)
durations = durs * 0.06
dur = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 60, 60+24)

amps = k.constant(dur, -44)

pitches = pad_to_dur(pitches, dur)

phrase = Phrase(pitches, amps)
part2.add_phrase(phrase)

silence_steps = int(silence_dur2*k.SECOND)
# part2.add_phrase(Phrase(np.zeros(silence_steps)+55, np.zeros(silence_steps)-100))
pitches = k.constant(silence_dur2, np.round(part2.phrases[-1].pitch[-1]))
amps = k.triangle(silence_dur2, 2/silence_dur2, 0.2, -90, -44)
part2.add_phrase(Phrase(pitches, amps))

# 82 pitches
pitches = k.fractal(new_seed2[::-1], 3)
durs = fib(len(pitches))
durs = np.roll(durs, 1)
durations = durs * 0.06
dur = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 62, 62+24)

amps = k.constant(dur, -44)

pitches = pad_to_dur(pitches, dur)

phrase = Phrase(pitches, amps)
part2.add_phrase(phrase)

n_steps = 10 * 4
pitches = k.fractal(new_seed, 1)
pitches = np.tile(pitches, 1 + n_steps//len(pitches))[:n_steps]
durs = fib(len(pitches))
durs = np.roll(durs, 1)
durations = durs * ritardando
dur = sum(durations)
pitches = k.figure(pitches, durations)
pitches = k.rescale(pitches, 62, 62+24)
pitches = pad_to_dur(pitches, dur)
pitches += k.sinusoid(dur, 4, 0, 0.25, np.random.uniform())

amps = k.constant(dur, -44)
human = np.random.normal(0, 0.125, len(amps)-1)
human = k.from_delta(human)
amps += human

phrase = Phrase(pitches, amps)
part2.add_phrase(phrase)

In [ ]:
part2.show()
part2.play()

In [ ]:
score = Score([part, part2, part3, part4])
score.save('fib', './audio-data/december/')

In [ ]:
parts = (vln1, vln2)

In [ ]:
figure, axes = plt.subplots(2)
print(type(axes))
for p in parts:
    fig = plt.figure(p.show())
    ax = fig.axes
    print(type(fig.axes))
    gs = ax[0, 1].get_gridspec()
    figure.add_subplot(gs)

In [ ]:
shit = vln1.show()
shit = plt.figure(shit)

In [ ]:
type(shit)